<b>
Note that this is meant as an introductory and not a comprehensive tutorial.  See the Weaviate documentation for more detail. 

https://weaviate.io/developers/weaviate/search
<b>

In [ ]:
!pip install weaviate-client --quiet

In [ ]:
import ast
import json
import pandas as pd
import weaviate

In [ ]:
wv_client = weaviate.Client(url="http://k8s-weaviate-weaviate-f1e22650f9-583768105cf15612.elb.us-west-2.amazonaws.com")

<h1>Basic Search</h1>

In [ ]:
response = (
    wv_client.query
    .get("Article", ['title','content','url','c_access'])
    .with_limit(1)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# return the vector associated with the obejct 
response = (
    wv_client.query
    .get("Article", ['title','content','url','c_access'])
    .with_additional("vector")
    .with_limit(1)
    .do()
)

print(json.dumps(response, indent=2))

<h1>Filtering based on metadata</h1>

In [ ]:
where_filter = {
    "path": ["c_access"],
    "operator": "Equal",
    "valueText": "france",
}

result = (
    wv_client.query
    .get("Article", ['title','content','url','c_access'])
    .with_where(where_filter)
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=2))

<h1>Aggregate data</h1>

In [ ]:
response = (
    wv_client.query
    .aggregate("Article")
    .with_meta_count()
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# combine metadata filtering and aggregation

where_filter = {
    "path": ["c_access"],
    "operator": "Equal",
    "valueText": "car",
}

response = (
    wv_client.query
    .aggregate("Article")
    .with_where(where_filter)
    .with_meta_count()
    .do()
)

print(json.dumps(response, indent=2))

<h1>Similarity / Vector Search</h1>

In [ ]:
nearText = {"concepts": ["modern art in Europe"]}

response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_near_text(nearText)
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# NOTE you cannot use "with_near_text" to query the class without a transformer, as it is unknown how to embed the text
# Instead, we can use "with_near_vector"

# collect a single vector
result = (
    wv_client.query
    .get("ArticleNoTransformer")
    .with_additional("vector")
    .with_limit(1)
    .do()
)

vector = result['data']['Get']['ArticleNoTransformer'][0]['_additional']['vector']

In [ ]:
# search for objects near this vector
nearVector = {"vector": vector}

response = (
    wv_client.query
    .get("ArticleNoTransformer", ['title','url'])
    .with_near_vector(nearVector)
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# IMPORTANT! The model used to embed your query must be the same as the model used to embed the underlying database (i.e. same vector length)
vector = vector[:10]
nearVector = {"vector": vector}

response = (
    wv_client.query
    .get("ArticleNoTransformer", ['title','url'])
    .with_near_vector(nearVector)
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# NOTE you can also use "with_near_vector" in the Class we've enabled Weaviate to create the embeddings

# collect a single vector
result = (
    wv_client.query
    .get("Article")
    .with_additional("vector")
    .with_limit(1)
    .do()
)

vector = result['data']['Get']['Article'][0]['_additional']['vector']

# query for objects near the vector
nearVector = {"vector": vector}

response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_near_vector(nearVector)
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# we can combine metadata filtering and semantic search
where_filter = {
    "path": ["c_access"],
    "operator": "Equal",
    "valueText": "car",
}

nearText = {"concepts": ["modern art in Europe"]}

result = (
    wv_client.query
    .get("Article", ['title','content','url'])
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(result, indent=2))

In [ ]:
# ... and we can set a threshold for the distance
where_filter = {
    "path": ["c_access"],
    "operator": "Equal",
    "valueText": "car",
}

max_distance = .25
nearText = {"concepts": ["modern art in Europe"],
           "distance": max_distance}


result = (
    wv_client.query
    .get("Article", ['title','content','url'])
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(result, indent=2))

<h1>BM25 / Keyword Search</h1>

In [ ]:
# basic bm25 search
response = (
    wv_client.query
    .get("Article", ['title','content','url','c_access'])
    .with_bm25(query="food")
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# return the bm25 algorithm score
response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_bm25(query="food")
    .with_limit(3)
    .with_additional("score")
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# perform bm25 search only in specific fields of the object
response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_bm25(query="food", properties=["url"]) # if not provided, all text fields are searched
    .with_limit(3)
    .with_additional("score")
    .do()
)

print(json.dumps(response, indent=2))

<h1>Hybrid (BM25 + Semantic Vector) Search</h1>

In [ ]:
# basic hybrid search
response = (
    wv_client.query
    .get("Article", ['title','url','c_access'])
    .with_hybrid(query="food")
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# include bm25 score and explanations
response = (
    wv_client.query
    .get("Article", ['title','url','c_access'])
    .with_hybrid(query="food")
    .with_additional(["score", "explainScore"])
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# weight the bm25 and vector scores
response = (
    wv_client.query
    .get("Article", ['title','url','c_access'])
    .with_hybrid(query="food", alpha=0.25) # An alpha of 1 is for a pure vector search and 0 is for a pure keyword search
    .with_additional(["score"])
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# note you can combine metadata filtering with all of these search types

where_filter = {
    "path": ["c_access"],
    "operator": "Equal",
    "valueText": "france",
}


response = (
    wv_client.query
    .get("Article", ['title','url', 'c_access'])
    .with_hybrid(query="food", alpha=0.25) # An alpha of 1 is for a pure vector search and 0 is for a pure keyword search
    .with_additional(["score"])
    .with_where(where_filter)
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

<h1>Reranking Search</h1>

In [ ]:
# Reranking vector search results
# let's say we want to find Q&As about Europe, and further sort towards the top those about capitals of countries.

# we can start with searching near the concept of Europe
nearText = {"concepts": ["Europe"]}

response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_near_text(nearText)
    .with_additional('distance')
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# and sort to the top those who's "content" fields matches the re-rank query for captial

nearText = {"concepts": ["Europe"]}

response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_near_text(nearText)
    .with_additional('rerank(property: "content" query: "capital") { score }')
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
# rerank bm25 search
response = (
    wv_client.query
    .get("Article", ['title','url'])
    .with_bm25(
      query='Europe'
    )
    .with_additional('rerank(property: "content" query: "capital") { score }')
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=2))